In [144]:
import pandas as pd
import warnings
from scipy.stats import zscore
from kestra import Kestra


warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'kestra'

In [ ]:
# Chargement des fichiers Excel
web = pd.read_excel("data/Fichier_web.xlsx", engine='openpyxl')
erp = pd.read_excel("data/Fichier_erp.xlsx", engine='openpyxl')
liaison = pd.read_excel("data/fichier_liaison.xlsx", engine='openpyxl')

In [76]:
print("BEFORE PROCESSING")
print("----------------------------")
print("the length of web is", len(web))
print("the length of erp is", len(erp))
print("the length of liaison is", len(liaison))

BEFORE PROCESSING
----------------------------
the length of web is 714
the length of erp is 825
the length of liaison is 825


In [78]:
print()
print("INITIAL PROCESSING")
print("----------------------------")

erp = erp.drop_duplicates(subset="product_id")
print("the length of erp after deduplication is", len(erp))
liaison = liaison.drop_duplicates()
print("the length of liaison after deduplication is", len(liaison))

# Nettoyage : suppression des lignes vides
web = web.dropna(subset=["sku", "post_title"])
print("the length of web after NA cleaning is", len(web))

# Supprimer les doublons
web = web.drop_duplicates(subset="sku")
print("the length of web after deduplication is", len(web))




INITIAL PROCESSING
----------------------------
the length of erp after deduplication is 825
the length of liaison after deduplication is 825
the length of web after NA cleaning is 714
the length of web after deduplication is 714


In [83]:
web.columns

Index(['sku', 'virtual', 'downloadable', 'rating_count', 'average_rating',
       'total_sales', 'tax_status', 'tax_class', 'post_author', 'post_date',
       'post_date_gmt', 'post_content', 'post_title', 'post_excerpt',
       'post_status', 'comment_status', 'ping_status', 'post_password',
       'post_name', 'post_modified', 'post_modified_gmt',
       'post_content_filtered', 'post_parent', 'guid', 'menu_order',
       'post_type', 'post_mime_type', 'comment_count'],
      dtype='object')

In [88]:
# Merge web and liaison using the 'sku' and 'id_web' columns
merged_df = pd.merge(web, liaison, left_on='sku', right_on='id_web', how='left')

# Merge the result with erp using the 'product_id' column
final_df = pd.merge(merged_df, erp, on='product_id', how='left')

# Drop the 'id_web' column if you don't need it anymore
final_df.drop(columns=['id_web'], inplace=True)


In [125]:
final_df["CA"] = final_df["price"] * final_df["total_sales"]
final_df["CA"].sum()

np.float64(65652.6)

In [126]:
final_df[["post_title", "price", "total_sales", "CA"]].sort_values("CA", ascending=False).head(10)


,post_title,price,total_sales,CA
526,Champagne Gosset Grand Blanc de Blancs,49.0,96.0,4704.0
319,Cognac Frapin VIP XO,176.0,13.0,2288.0
572,Champagne Gosset Grand Millésime 2006,53.0,30.0,1590.0
530,Champagne Gosset Grande Réserve,39.0,40.0,1560.0
656,Champagne Egly-Ouriet Grand Cru Blanc de Noirs,126.5,11.0,1391.5
701,Champagne Egly-Ouriet Grand Cru Millésimé 2008,225.0,5.0,1125.0
20,Elian Daros Côtes du Marmandais Clos Baquey 2015,29.0,36.0,1044.0
421,Domaine Giudicelli Patrimonio Blanc 2019,25.2,41.0,1033.2
587,Clos du Mont-Olivet Châteauneuf-du-Pape 2012,44.3,23.0,1018.9
49,Marc Colin Et Fils Chassagne-Montrachet Blanc ...,43.9,23.0,1009.7


In [132]:
mean_price = final_df["price"].mean()
std_price = final_df["price"].std()

# Calculate the z-score for each wine
final_df["z_score"] = (final_df["price"] - mean_price) / std_price

# Identify millésime and ordinary wines
final_df["type_vin"] = final_df["z_score"].apply(lambda z: "millésime" if z > 2 else "ordinaire")

# Check the result
final_df[["post_title", "price", "z_score", "type_vin"]].sort_values("z_score", ascending=False)[1:32]

,post_title,price,z_score,type_vin
264,David Duband Charmes-Chambertin Grand Cru 2014,217.5,6.652405,millésime
678,Coteaux Champenois Egly-Ouriet Ambonnay Rouge ...,191.3,5.710315,millésime
319,Cognac Frapin VIP XO,176.0,5.160164,millésime
306,Camille Giroud Clos de Vougeot 2016,175.0,5.124206,millésime
383,Cognac Frapin Château de Fontpinot 1989 20 Ans...,157.0,4.476969,millésime
382,Domaine Des Croix Corton Charlemagne Grand Cru...,137.0,3.757817,millésime
682,Champagne Gosset Célébris Vintage 2007,135.0,3.685902,millésime
656,Champagne Egly-Ouriet Grand Cru Blanc de Noirs,126.5,3.380262,millésime
310,Domaine Weinbach Gewurztraminer Grand Cru Furs...,124.8,3.319134,millésime
388,Wemyss Malts Single Cask Scotch Whisky Choc 'n...,122.0,3.218453,millésime


In [143]:
final_df[final_df["type_vin"] == "millésime"].shape[0]

30

In [137]:
# Select only the relevant columns for the first sheet
wine_details = final_df[[
    "sku", "post_title", "price", "total_sales", "post_date", "CA"
]].sort_values("CA", ascending=False)
# Create a one-row DataFrame for the summary sheet
total_ca_df = pd.DataFrame({
    "Total Chiffre d'Affaires (€)": [final_df["CA"].sum()]
})

with pd.ExcelWriter("output/rapport_chiffres_affaires.xlsx") as writer:
    # Write the detailed wine data
    wine_details.to_excel(writer, sheet_name="Détails des Vins", index=False)
    
    # Write the total CA
    total_ca_df.to_excel(writer, sheet_name="Total CA", index=False)


In [140]:
# Extract only the premium wines (z-score > 2)
premium_wines = final_df[final_df["type_vin"] == "millésime"][[
    "sku", "post_title", "post_excerpt","price", "post_date"]]
premium_wines.to_csv("output/vins_millesimes.csv", index=False)


In [141]:
# Extract only the premium wines (z-score under 2)
premium_wines = final_df[final_df["type_vin"] == "ordinaire"][[
    "sku", "post_title", "post_excerpt","price", "post_date"]]
premium_wines.to_csv("output/vins_ordinaire.csv", index=False)


In [68]:
# Sauvegarde en Excel
with pd.ExcelWriter("rapport_vins.xlsx") as writer:
    ca_par_produit.to_excel(writer, sheet_name="CA_par_produit", index=False)
    pd.DataFrame([{"CA_total": ca_total}]).to_excel(writer, sheet_name="CA_total", index=False)
    vins_premium.to_excel(writer, sheet_name="Vins_premium", index=False)
    vins_ordinaires.to_excel(writer, sheet_name="Vins_ordinaires", index=False)


In [71]:
assert not web.isnull().any().any(), "Valeurs manquantes dans 'web'"
assert not erp.isnull().any().any(), "Valeurs manquantes dans 'erp'"
assert df["CA"].min() >= 0, "CA négatif détecté"
assert "premium" in df["type_vin"].values, "Aucun vin premium détecté"


AssertionError: Valeurs manquantes dans 'web'